In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:

import pandas_datareader as web
import datetime as dt

In [ ]:
import keras
import tensorflow as tf
from keras.models import Model
from keras.layers import Dense, Dropout, LSTM, Input, Activation, concatenate
from keras import optimizers
import time

In [ ]:
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, LSTM
from statsmodels.tsa.seasonal import seasonal_decompose

In [ ]:
df1=pd.read_csv("/kaggle/input/audusd-csv-dataset/AUDUSD_M1_201501020900_202108202354.csv",sep='\t')

In [ ]:
df1.head()

In [ ]:
df1.shape

In [ ]:
df1.info()


To find out the null values

In [ ]:
sns.heatmap(df1.isnull(),yticklabels=False)

No NaN values

In [ ]:
%matplotlib inline 
plt.gcf().set_size_inches(20, 10, forward=True)
plt.plot(df1['<CLOSE>'], color="black")

plt.title(" Share Price")
plt.xlabel("Days")
plt.ylabel("Share Price")

plt.show()

In [ ]:
df1["DATE"] = df1["<DATE>"] +" " +df1["<TIME>"]

In [ ]:
df1.head()

In [ ]:
df1['DATE'] = df1['DATE'].astype('datetime64[ns]')

In [ ]:
df1.info()

In [ ]:
#df1["DATE"]=pd.to_datetime(df1["<DATE>"])
#df1["TIME"]=pd.to_datetime(df1["<TIME>"])

In [ ]:
df1.set_index("DATE", inplace=True)

In [ ]:
%matplotlib inline 
plt.gcf().set_size_inches(20, 10, forward=True)
plt.plot(df1['<CLOSE>'], color="black")

plt.title(" Share Price")
plt.xlabel("Days")
plt.ylabel("Share Price")

plt.show()

In [ ]:
#df1['<TIME>'] = df1['<TIME>'].astype('datetime64[ns]')

In [ ]:
#df1.info()

In [ ]:
df1.head()

In [ ]:
df1["DATE1"] = df1["<DATE>"] +" " +df1["<TIME>"]
df1['DATE1'] = df1['DATE1'].astype('datetime64[ns]')
df2=df1[df1["DATE1"].dt.year == 2015]

In [ ]:
df2.shape

In [ ]:
df3=df1[df1['DATE1'].dt.year == 2016]

In [ ]:
df3.shape

In [ ]:
df4=df1[df1['DATE1'].dt.year == 2017]

In [ ]:
k=int(df4.shape[0]/4)
df5=df4[:k]
df5.shape

In [ ]:
df5.head()

In [ ]:
vertical_stack = pd.concat([df2, df3,df5], axis=0)


In [ ]:
def Rsi_finder(series, period):
    delta = series.diff().dropna()
    u = delta * 0
    d = u.copy()
    u[delta > 0] = delta[delta > 0]
    d[delta < 0] = -delta[delta < 0]
    u[u.index[period-1]] = np.mean( u[:period] ) #first value is sum of avg gains
    u = u.drop(u.index[:(period-1)])
    d[d.index[period-1]] = np.mean( d[:period] ) #first value is sum of avg losses
    d = d.drop(d.index[:(period-1)])
    rs = pd.DataFrame.ewm(u, com=period-1, adjust=False).mean() / \
         pd.DataFrame.ewm(d, com=period-1, adjust=False).mean()
    return 100 - 100 / (1 + rs)

In [ ]:
vertical_stack['RSI'] = Rsi_finder(vertical_stack["<CLOSE>"], 14)

In [ ]:
for i in range(14):
  vertical_stack["RSI"][i]=vertical_stack["RSI"][15]
vertical_stack.head()

In [ ]:
vertical_stack.drop(['<TIME>',"<DATE>","<VOL>","DATE1"], axis = 1,inplace=True)

In [ ]:
def calculate_sma(data, n):
    sma = data.rolling(window = n).mean()
    return pd.DataFrame(sma)

In [ ]:
vertical_stack["sma_30"] = calculate_sma(vertical_stack['<CLOSE>'],30)
for i in range(29):
  vertical_stack["sma_30"][i]=vertical_stack['sma_30'][30]

In [ ]:
def calculate_ema(prices, days, smoothing=2):
    ema = [sum(prices[:days]) / days]
    for price in prices[days:]:
        ema.append((price * (smoothing / (1 + days))) + ema[-1] * (1 - (smoothing / (1 + days))))
    return pd.DataFrame(ema)

In [ ]:
vertical_stack["ema_10"] = calculate_ema(vertical_stack['<CLOSE>'],10)
for i in range(9):
    vertical_stack["ema_10"][i]=vertical_stack['ema_10'][10]

In [ ]:
vertical_stack.tail()
vertical_stack['1ema_10'] =vertical_stack['<CLOSE>'].ewm(span=10, adjust=False).mean()

In [ ]:
vertical_stack.tail()
vertical_stack.drop(["ema_10"], axis = 1,inplace=True)

In [ ]:
plt.gcf().set_size_inches(22, 10, forward=True)

start = 0
end = 10000
#actual=plt.plot(actual[start+60:end], label= "actual")
real = plt.plot(vertical_stack['sma_30'][start:end], label='sma', color="red")
pred = plt.plot(vertical_stack['1ema_10'][start:end], label='ema', color="black")

plt.legend(['sma_30', 'ema_10'])
plt.title(" Share Price")
plt.xlabel("minutes")
plt.ylabel("Share Price")
plt.show()

In [ ]:
vertical_stack.info()

In [ ]:
#vertical_stack.set_index('DATE', inplace=True)
#vertical_stack.info()

In [ ]:
from sklearn import preprocessing
data_normaliser = preprocessing.StandardScaler()
df1_normalised = data_normaliser.fit_transform(vertical_stack)

In [ ]:
df1_normalised[0,:]

In [ ]:
history_points=60

In [ ]:
ohlcv_histories_normalised = np.array([df1_normalised[i  : i + history_points].copy() for i in range(len(df1_normalised) - history_points)])

In [ ]:
weighted_avg_values_normalised = np.array([df1_normalised[:,3][i + history_points].copy() for i in range(len(df1_normalised) - history_points)])

In [ ]:
print(weighted_avg_values_normalised.shape)
weighted_avg_values_normalised = np.expand_dims(weighted_avg_values_normalised, -1)
weighted_avg_values_normalised.shape

In [ ]:
weighted_avg_values = np.array([vertical_stack["<CLOSE>"][i + history_points].copy() for i in range(len(vertical_stack) - history_points)])

In [ ]:
y_normaliser = preprocessing.StandardScaler()
y_normaliser.fit(np.expand_dims( weighted_avg_values,-1 ))

In [ ]:
print(weighted_avg_values.shape)

In [ ]:
test_split = 0.89 # the percent of data to be used for testing
n = int(ohlcv_histories_normalised.shape[0] * test_split)
n

In [ ]:
x_train = ohlcv_histories_normalised[:n]
print(x_train.shape)

In [ ]:
y_train = weighted_avg_values_normalised[:n]
y_train.reshape(n,1)
print(y_train.shape)

In [ ]:
x_test = ohlcv_histories_normalised[n:]
y_test = weighted_avg_values_normalised[n:]

In [ ]:
corrMatx = vertical_stack.corr()
corrMatx

In [ ]:
tpu = tf.distribute.cluster_resolver.TPUClusterResolver.connect()
tpu_strategy = tf.distribute.experimental.TPUStrategy(tpu)

In [ ]:
with tpu_strategy.scope():
    start = time.process_time()
    model21=Sequential()
    model21.add(LSTM(units=100, return_sequences=True, input_shape=(history_points,9)))
    model21.add(Dropout(0.20))
    model21.add(LSTM(units=100, return_sequences=True))
    model21.add(Dropout(0.20)) 
    model21.add(LSTM(units=100, return_sequences=True))
    model21.add(Dropout(0.20)) 
    model21.add(LSTM(units=100))
    model21.add(Dropout(0.20)) 
    model21.add(Dense(units=1))
    model21.compile(optimizer="adam", loss="mean_squared_error")

In [ ]:
model21.fit(x_train,y_train,batch_size=128, epochs=100) 
print(time.process_time() - start)

In [ ]:
y_test_predicted21 = model21.predict(x_test)

In [ ]:
y_test= y_normaliser.inverse_transform(y_test)
y_test

In [ ]:
y_test_predicted21 = y_normaliser.inverse_transform(y_test_predicted21)
y_test_predicted21

In [ ]:
plt.gcf().set_size_inches(22, 10, forward=True)

start = 0
end = -1
#actual=plt.plot(actual[start+60:end], label= "actual")
real = plt.plot(y_test[start:end], label='real', color="red")
pred = plt.plot(y_test_predicted21[start:end], label='predicted', color="black")

plt.legend(['Real', 'Predicted'])
plt.title(" Share Price")
plt.xlabel("minutes")
plt.ylabel("Share Price")
plt.show()

In [ ]:
from sklearn.metrics import mean_squared_error
mean_squared_error(y_test, y_test_predicted21)

In [ ]:
def percentage_error(actual, predicted):
    res = np.empty(actual.shape)
    for j in range(actual.shape[0]):
        if actual[j] != 0:
            res[j] = (actual[j] - predicted[j]) / actual[j]
        else:
            res[j] = predicted[j] / np.mean(actual)
    return res

def mean_absolute_percentage_error(y_true, y_pred): 
    return np.mean(np.abs(percentage_error(np.asarray(y_true), np.asarray(y_pred)))) * 100

In [ ]:
mean_absolute_percentage_error(y_test, y_test_predicted21)